# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: Alberto Renteria Camacho

**Professor**: Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-ALBERTO-RENTERIA") \
    .master("spark://70f60bba1584:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 13:30:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read the Data (10 points):
    Load the employees.csv and departments.csv files into PySpark DataFrames.

In [17]:
from team_name.spark_utils import SparkUtils

employees_schema = SparkUtils.generate_schema([("employee_id", "integer"), ("employee_info", "string")])

employees_df = spark.read \
                .schema(employees_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/employees.csv")

employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)



In [18]:
from team_name.spark_utils import SparkUtils

departments_schema = SparkUtils.generate_schema([("department_id", "integer"), ("department_name", "string"), ("location", "string")])

departments_df = spark.read \
                .schema(departments_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/departments.csv")

departments_df.printSchema()

root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)



Extract the employee info from the JSON column (20 points):
    Extract the following the columns: name (string), department_id (integer), salary (double), and hire_date (date) from the employee_info column

In [21]:
from pyspark.sql.functions import get_json_object, col
from pyspark.sql.types import IntegerType, DoubleType, DateType

employees_df = employees_df.withColumn("name", get_json_object(col("employee_info"), "$.name")) \
    .withColumn("department_id", get_json_object(col("employee_info"), "$.department_id").cast(IntegerType())) \
    .withColumn("salary", get_json_object(col("employee_info"), "$.salary").cast(DoubleType())) \
    .withColumn("hire_date", get_json_object(col("employee_info"), "$.hire_date").cast(DateType()))


Join Operations (10 points):
    Join the employees DataFrame with the departments DataFrame on department_id to enrich the employee data with department details.

In [ ]:
employees_info_df = employees_df.join(departments_df, employees_df["department_id"] == departments_df["department_id"], "left")